#Projeto de data sciencies em texto não supervisionado


# NIPS: Visualização de modelagem de tópicos

Alguns tópicos principais do NIPS de acordo com [wikipedia] (https://en.wikipedia.org/wiki/Conference_on_Neural_Information_Processing_Systems):

1. Aprendizado de máquina,
2. Estatísticas,
3. Inteligência artificial,
4. Neurociência computacional

No entanto, os tópicos estão dentro do mesmo domínio, o que torna mais difícil distingui-los. Aqui neste Kernel tentarei extrair alguns tópicos usando a alocação de Dirichlet latente __LDA__. Este tutorial apresenta um pipeline de processamento de linguagem natural de ponta a ponta, começando com dados brutos e passando pela preparação, modelagem e visualização do papel. Iremos abordar os seguintes pontos


1. Modelagem de tópico com ** LDA **
1. Visualização de modelos de tópicos com ** pyLDAvis **
1. Visualização dos resultados do LDA com ** t-SNE ** e ** bokeh **

In [1]:
%pylab inline

import pandas as pd
import pickle as pk
from scipy import sparse as sp

Populating the interactive namespace from numpy and matplotlib


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
workdir_path = '/content/drive/My Drive/'  # Inserir o local da pasta onde estão os arquivos de entrada (treino e teste)
os.chdir(workdir_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
p_df = pd.read_csv('papers/Papers.csv')
docs = array(p_df['PaperText'])
docs[1]

'Learning with Symmetric Label Noise: The\nImportance of Being Unhinged\n\nBrendan van Rooyen∗,†\n∗\n\nAditya Krishna Menon†,∗\n\nThe Australian National University\n\n†\n\nRobert C. Williamson∗,†\n\nNational ICT Australia\n\n{ brendan.vanrooyen, aditya.menon, bob.williamson }@nicta.com.au\n\nAbstract\nConvex potential minimisation is the de facto approach to binary classification.\nHowever, Long and Servedio [2010] proved that under symmetric label noise\n(SLN), minimisation of any convex potential over a linear function class can result in classification performance equivalent to random guessing. This ostensibly\nshows that convex losses are not SLN-robust. In this paper, we propose a convex,\nclassification-calibrated loss and prove that it is SLN-robust. The loss avoids the\nLong and Servedio [2010] result by virtue of being negatively unbounded. The\nloss is a modification of the hinge loss, where one does not clamp at zero; hence,\nwe call it the unhinged loss. We show that the o

## Primeiro pré-processar deixando todo em minuscula e tokenizar o texto

In [6]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [7]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = str(docs[idx]).lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isdigit()] for doc in docs]
    
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs

In [8]:
docs = docs_preprocessor(docs)

### **Computar bigramas e trigramas :**
Sine tópicos são muito semelhantes, o que os faria distingui-los são frases ao invés de palavras únicas / individuais.

In [9]:
from gensim.models import Phrases
# Add bigrams and trigrams to docs (only ones that appear 10 times or more).
bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
print(bigram)

Phrases<556123 vocab, min_count=10, threshold=10.0, max_vocab_size=40000000>


In [ ]:
print(trigram)

Phrases<616916 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>


### Remover Tokens comuns e pouco comuns**

In [10]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 39534
Number of unique words after removing rare and common words: 6001


Eliminando as palavras comuns e raras, acabamos com apenas cerca de 6% das palavras.

** Vetorizar dados: **
A primeira etapa é obter uma representação por trás das palavras de cada documento.
Passos
1-converter um corpus
doc2bow:converter documento (uma lista de palavras) no formato de saco de palavras = lista de (token_id, token_count) 2-tuplas. Cada palavra é considerada uma string tokenizada e normalizada (codificada em unicode ou em utf8). Nenhum outro pré-processamento é feito nas palavras do documento; aplique tokenização, lematização etc. antes de chamar esse método.


In [12]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [13]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 6001
Number of documents: 403




Com o corpus de bag of words, podemos prosseguir para aprender nosso modelo de tópico a partir dos documentos.

# Entrenando LDA

In [14]:
from gensim.models import LdaModel

In [15]:
# Set training parameters.
num_topics = 4
chunksize = 500 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 35.6 s, sys: 14.7 ms, total: 35.6 s
Wall time: 35.7 s


# Como escolher a quantidade de tópicos?
__LDA__ é uma técnica não supervisionada, o que significa que não sabemos antes de executar o modelo quantos tópicos existem em nosso corpus. A coerência do tópico é uma das principais técnicas utilizadas para desestimar o número de tópicos. Você pode ler sobre isso [aqui.] (Http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf)

No entanto, usei a ferramenta de visualização LDA ** pyLDAvis **, tentei alguns tópicos e comparei os resultados. Quatro pareciam ser o número ideal de tópicos que separariam mais os tópicos.

In [16]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.7MB/s 
     |████████████████████████████████| 552kB 18.4MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=aad01083a6e9e8bbcc6fe87112c96226cb0b22adfb79bc2d21651307d8d855f3
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=18091c0f28387428998261219458dcc36f8f71e1e6781c92453098db2e6cf435
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [18]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


In [19]:
pyLDAvis.gensim.prepare(model, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.130500 -0.094436       1        1  32.010582
3      0.005061  0.043572       2        1  25.281895
1      0.179014 -0.042217       3        1  22.834398
0     -0.053575  0.093081       4        1  19.873125, topic_info=                  Term         Freq        Total Category  logprob  loglift
609             regret  1186.000000  1186.000000  Default  30.0000  30.0000
3230  gaussian_process   899.000000   899.000000  Default  29.0000  29.0000
2180     convolutional   653.000000   653.000000  Default  28.0000  28.0000
3048  convergence_rate   935.000000   935.000000  Default  27.0000  27.0000
3287        submodular   583.000000   583.000000  Default  26.0000  26.0000
...                ...          ...          ...      ...      ...      ...
29              belief   229.866821   418.006561   Topic4  -5.9429   1.0178
1073             gibbs   226.657501   430.389893   Topic4  -5.9570   0.9746
7                agent   209.414490   482.046814   Topic4  -6.0361   0.7821
845       markov_chain   222.091339   813.919800   Topic4  -5.9773   0.3170
2511              item   204.261658   602.441284   Topic4  -6.0610   0.5342

[291 rows x 6 columns], token_table=      Topic      Freq                  Term
term                                       
1475      1  0.982718     absolute_constant
1475      4  0.014242     absolute_constant
4737      1  0.979575  accelerated_gradient
3185      1  0.012290       active_learning
3185      2  0.898676       active_learning
...     ...       ...                   ...
2132      1  0.065538        weighted_graph
2132      2  0.010923        weighted_graph
2132      3  0.010923        weighted_graph
2132      4  0.917537        weighted_graph
3302      4  0.977421                wolsey

[821 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 2, 1])

** O que vemos aqui? **

** O painel esquerdo **, rotulado Mapa de distância intertópica, os círculos representam diferentes tópicos e a distância entre eles. Tópicos semelhantes aparecem mais próximos e tópicos diferentes mais distantes.
O tamanho relativo do círculo de um tópico no gráfico corresponde à frequência relativa do tópico no corpus.
Um tópico individual pode ser selecionado para um exame mais detalhado clicando em seu círculo ou inserindo seu número na caixa "tópico selecionado" no canto superior esquerdo.
 
** O painel direito ** inclui o gráfico de barras dos 30 principais termos. Quando nenhum tópico é selecionado no gráfico à esquerda, o gráfico de barras mostra os 30 termos mais "salientes" no corpus. A saliência de um termo é uma medida de quão frequente o termo é no corpus e quão "distinto" ele é na distinção entre diferentes tópicos.
Selecionar cada tópico à direita modifica o gráfico de barras para mostrar os termos "relevantes" para o tópico selecionado.
A relevância é definida como no rodapé 2 e pode ser ajustada pelo parâmetro $ \ lambda $, menor $ \ lambda $ dá maior peso à distinção do termo, enquanto $ \ lambda $ s maior corresponde à probabilidade da ocorrência do termo por tópicos.

Portanto, para ter uma noção melhor dos termos por tópico, usaremos $ \ lambda $ = 0.

** Como avaliar nosso modelo? **
Então, novamente, uma vez que não há base para aqui, temos que ser criativos na definição de maneiras de avaliar. Eu faço isso em duas etapas:

1. divida cada documento em duas partes e veja se os tópicos atribuídos a eles são semelhantes. => quanto mais semelhante, melhor
2. comparar documentos escolhidos aleatoriamente entre si. => quanto menos semelhante melhor

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

p_df['tokenz'] = docs

docs1 = p_df['tokenz'].apply(lambda l: l[:int0(len(l)/2)])
docs2 = p_df['tokenz'].apply(lambda l: l[int0(len(l)/2):])

transformando os dados

In [21]:
corpus1 = [dictionary.doc2bow(doc) for doc in docs1]
corpus2 = [dictionary.doc2bow(doc) for doc in docs2]

# Usando a transformação do modelo LDA corpus
lda_corpus1 = model[corpus1]
lda_corpus2 = model[corpus2]

In [22]:
from collections import OrderedDict
def get_doc_topic_dist(model, corpus, kwords=False):
    
    '''
   A transformação LDA, para cada doc retorna apenas tópicos com peso diferente de zero
     Esta função faz uma transformação de matriz de documentos no espaço do tópico.
    '''
    top_dist =[]
    keys = []

    for d in corpus:
        tmp = {i:0 for i in range(num_topics)}
        tmp.update(dict(model[d]))
        vals = list(OrderedDict(tmp).values())
        top_dist += [array(vals)]
        if kwords:
            keys += [array(vals).argmax()]

    return array(top_dist), keys

In [23]:
top_dist1, _ = get_doc_topic_dist(model, lda_corpus1)
top_dist2, _ = get_doc_topic_dist(model, lda_corpus2)

print("Intra similarity: cosine similarity for corresponding parts of a doc(higher is better):")
print(mean([cosine_similarity(c1.reshape(1, -1), c2.reshape(1, -1))[0][0] for c1,c2 in zip(top_dist1, top_dist2)]))

random_pairs = np.random.randint(0, len(p_df['PaperText']), size=(400, 2))

print("Inter similarity: cosine similarity between random parts (lower is better):")
print(np.mean([cosine_similarity(top_dist1[i[0]].reshape(1, -1), top_dist2[i[1]].reshape(1, -1)) for i in random_pairs]))

Intra similarity: cosine similarity for corresponding parts of a doc(higher is better):
0.8952567
Inter similarity: cosine similarity between random parts (lower is better):
0.418154


## Vejamos os termos que aparecem mais em cada tópico.

In [24]:
def explore_topic(lda_model, topic_number, topn, output=True):
    """
    accept a ldamodel, atopic number and topn vocabs of interest
    prints a formatted list of the topn terms
    """
    terms = []
    for term, frequency in lda_model.show_topic(topic_number, topn=topn):
        terms += [term]
        if output:
            print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))
    
    return terms

In [25]:
topic_summaries = []
print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')
for i in range(num_topics):
    print('Topic '+str(i)+' |---------------------\n')
    tmp = explore_topic(model,topic_number=i, topn=10, output=True )
#     print tmp[:5]
    topic_summaries += [tmp[:5]]
    print

term                 frequency

Topic 0 |---------------------

vertex               0.007
submodular           0.007
random_walk          0.006
graphical_model      0.005
greedy               0.005
tensor               0.005
submodular_function  0.004
data_set             0.004
greedy_algorithm     0.004
polynomial_time      0.004
Topic 1 |---------------------

convolutional        0.006
fully_connected      0.005
deep_learning        0.005
recurrent            0.005
recurrent_neural     0.005
hidden_unit          0.005
hidden_layer         0.005
lstm                 0.005
embedding            0.005
pixel                0.004
Topic 2 |---------------------

regret               0.008
convergence_rate     0.006
matrix_completion    0.005
step_size            0.005
sample_complexity    0.005
rank_matrix          0.005
bandit               0.005
gradient_descent     0.004
singular_value       0.004
strongly_convex      0.003
Topic 3 |---------------------

gaussian_process     0.008
var


De cima, é possível inspecionar cada tópico e atribuir um rótulo interpretável por humanos a ele. Aqui, eu os rotulei da seguinte maneira:

In [26]:
top_labels = {0: 'Statistics', 1:'Numerical Analysis', 2:'Online Learning', 3:'Deep Learning'}

In [27]:
import re
import nltk
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stops = set(stopwords.words('english'))

def paper_to_wordlist( paper, remove_stopwords=True ):
    '''
        Function converts text to a sequence of words,
        Returns a list of words.
    '''
    lemmatizer = WordNetLemmatizer()
    # 1. Remove non-letters
    paper_text = re.sub("[^a-zA-Z]"," ", paper)
    # 2. Convert words to lower case and split them
    words = paper_text.lower().split()
    # 3. Remove stop words
    words = [w for w in words if not w in stops]
    # 4. Remove short words
    words = [t for t in words if len(t) > 2]
    # 5. lemmatizing
    words = [nltk.stem.WordNetLemmatizer().lemmatize(t) for t in words]

    return(words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


###Agora vamos mudar a representação TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvectorizer = TfidfVectorizer(input='content', analyzer = 'word', lowercase=True, stop_words='english',\
                                  tokenizer=paper_to_wordlist, ngram_range=(1, 3), min_df=40, max_df=0.20,\
                                  norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=True)

dtm = tvectorizer.fit_transform(p_df['PaperText']).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
top_dist =[]
for d in corpus:
    tmp = {i:0 for i in range(num_topics)}
    tmp.update(dict(model[d]))
    vals = list(OrderedDict(tmp).values())
    top_dist += [array(vals)]

In [ ]:
top_dist, lda_keys= get_doc_topic_dist(model, corpus, True)
features = tvectorizer.get_feature_names()

In [ ]:
top_ws = []
for n in range(len(dtm)):
    inds = int0(argsort(dtm[n])[::-1][:4])
    tmp = [features[i] for i in inds]
    
    top_ws += [' '.join(tmp)]
    
p_df['Text_Rep'] = pd.DataFrame(top_ws)
p_df['clusters'] = pd.DataFrame(lda_keys)
p_df['clusters'].fillna(10, inplace=True)

cluster_colors = {0: 'blue', 1: 'green', 2: 'yellow', 3: 'red', 4: 'skyblue', 5:'salmon', 6:'orange', 7:'maroon', 8:'crimson', 9:'black', 10:'gray'}

p_df['colors'] = p_df['clusters'].apply(lambda l: cluster_colors[l])

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(top_dist)

In [ ]:
p_df['X_tsne'] =X_tsne[:, 0]
p_df['Y_tsne'] =X_tsne[:, 1]

In [ ]:
from bokeh.plotting import figure, show, output_notebook, save#, output_file
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource
output_notebook()

In [ ]:
source = ColumnDataSource(dict(
    x=p_df['X_tsne'],
    y=p_df['Y_tsne'],
    color=p_df['colors'],
    label=p_df['clusters'].apply(lambda l: top_labels[l]),
#     msize= p_df['marker_size'],
    topic_key= p_df['clusters'],
    title= p_df[u'Title'],
    content = p_df['Text_Rep']
))

In [ ]:
title = 'T-SNE visualization of topics'

plot_lda = figure(plot_width=1000, plot_height=600,
                     title=title, tools="pan,wheel_zoom,box_zoom,reset,hover",
                     x_axis_type=None, y_axis_type=None, min_border=1)

plot_lda.scatter(x='x', y='y', legend='label', source=source,
                 color='color', alpha=0.8, size=10)#'msize', )

# hover tools
hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips = {"content": "Title: @title, KeyWords: @content - Topic: @topic_key "}
plot_lda.legend.location = "top_left"

show(plot_lda)

#save the plot
# save(plot_lda, '{}.html'.format(title))

#Agora temos labels para nossos grupos 